In [21]:
# Runs in Google Colab

# Install packages
# !pip install -q google-auth requests pandas

# Auth set up
from google.colab import auth
auth.authenticate_user()

# Get access token
import google.auth
from google.auth.transport.requests import Request

creds, detected_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/bigquery"]
)
creds.refresh(Request())
access_token = creds.token

# Call BigQuery via REST, then convert JSON to CSV
import json
import requests
import pandas as pd
import os

PROJECT_ID = "cis9440-indv-assignment"
DATASET_ID = "nyc_mvc_star"

def bq_to_csv(
    output_csv: str = "bq_mvc_export.csv",
    limit_rows: int = 200
):
    url = f"https://bigquery.googleapis.com/bigquery/v2/projects/{PROJECT_ID}/queries"

    sql = f"""
        SELECT
          d.calendar_date,
          l.borough,
          l.zip_code,
          f.number_of_persons_injured,
          f.number_of_persons_killed
        FROM `{PROJECT_ID}.{DATASET_ID}.fact_collision` f
        JOIN `{PROJECT_ID}.{DATASET_ID}.dim_date` d
          ON f.date_key = d.date_key
        JOIN `{PROJECT_ID}.{DATASET_ID}.dim_location` l
          ON f.location_key = l.location_key
        LIMIT {limit_rows}
    """

    body = {
        "query": sql,
        "useLegacySql": False
    }

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    resp = requests.post(url, headers=headers, data=json.dumps(body))
    resp.raise_for_status()

    data = resp.json()

    fields = data.get("schema", {}).get("fields", [])
    rows_json = data.get("rows", [])

    if not fields:
        print("Invalid. No schema returned")
        return None

    if not rows_json:
        print("Invalid. No rows returned at all")
        return None

    columns = [f["name"] for f in fields]
    rows = [
        [cell["v"] for cell in row["f"]]
        for row in rows_json
    ]

    # build DataFrame
    df = pd.DataFrame(rows, columns=columns)

    # write CSV
    df.to_csv(output_csv, index=False)

    print(f"Wrote {len(df)} rows to {output_csv}")
    display(df.head())

    return df

# Run export
df_result = bq_to_csv(
    output_csv="bq_mvc_export.csv",
    limit_rows=200
)

# Show working directory for refernce
print("\nCurrent working folder:", os.getcwd())
print("Files here:", os.listdir())

Wrote 200 rows to bq_mvc_export.csv


,calendar_date,borough,zip_code,number_of_persons_injured,number_of_persons_killed
0,2022-03-26,Manhattan,10032.0,1,0
1,2021-12-09,Queens,11102.0,1,0
2,2021-04-14,Bronx,10464.0,1,0
3,2021-07-04,Queens,11369.0,1,0
4,2021-03-28,Brooklyn,11221.0,1,0



Current working folder: /content
Files here: ['.config', 'bq_mvc_export.csv', '.ipynb_checkpoints', 'sample_data']
